<a href="https://colab.research.google.com/github/sharmaar342/sharmaar342/blob/main/Python_Boot_Camp_Lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Boot Camp Lesson 3

**Author:** Nicholas Colella<br>
**Date created:** 2021/08/15<br>
**Last modified:** 2021/01/24<br>


# Course Information

You are encouraged to watch the corresponding video, available on Canvas, as you work through this notebook. 

Additionally, we strongly encourage you to test your understanding of the material as you go! The Canvas quiz can be completed while you watch the video and work on the notebook (and can be taken multiple times).

# Pandas DataFrames

Recall from the last lesson we created a NumPy array then saved it to a .csv file.

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
my_matrix_1 = np.array([[1, 2, 3], [4, 5, 6]])
print(my_matrix_1)

np.savetxt('./gdrive/MyDrive/my_array.csv', my_matrix_1, delimiter=',')
loaded_array = np.loadtxt('./gdrive/MyDrive/my_array.csv', delimiter=',')
loaded_array

Oftentimes we have labeled data, and we want to present our data more cleanly in a spreadsheet-type manner. This is similar to working with Excel. For this, we use pandas in Python.

In [ ]:
pd.DataFrame(loaded_array)

We can also load csv files directly.

In [ ]:
pd.read_csv('./gdrive/MyDrive/my_array.csv') # this assumes our first row is the header!

In [ ]:
my_df = pd.read_csv('./gdrive/MyDrive/my_array.csv', header=None) # generates a header for us
my_df

We can rename our rows (indices) and columns.

In [ ]:
my_df.columns = ['first col', 'second col', 'third col'] # we can also assign these during import with read_csv, see https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
my_df.index = ['first row', 'second row'] # Can be pulled with read_csv if present in file, see https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html
my_df

We can now access data either with the given **name** with `.loc` or by the numeric **index** with `.iloc`.

Specific point:

In [ ]:
print(my_df.loc['first row', 'second col'])
print(my_df.iloc[0, 1])
print(my_df.iloc[0][1])

A given column:

In [ ]:
print(my_df['second col'])

In [ ]:
print(my_df.loc[:, 'second col'])

In [ ]:
print(my_df.iloc[:, 1])

In [ ]:
# print(my_df.iloc[:][1]) # Errors, can't use this

A given row:

In [ ]:
print(my_df.loc['first row'])

In [ ]:
print(my_df.loc['first row', :])

In [ ]:
print(my_df.iloc[0])

In [ ]:
print(my_df.iloc[0, :])

# Data

While you can upload data to Colab directly or via Google Drive, you can also pull data from online sources. We can use `wget` to download a zip file from a website to our Colab runtime. The `!` before `wget` signifies that this is a bash command (usually executed in terminal), *not* a Python command. Similarly, we can use the bash command `unzip` to unzip the file, again using `!` to signify that it should be run in bash, not Python.

In [ ]:
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

Now that we have unzipped the data, we can load it into a Pandas dataframe called `weather_df` with the function `pd.read_csv`.

In [ ]:
weather_df = pd.read_csv('jena_climate_2009_2016.csv')
weather_df.head()

We can also open online files directly with the urllib request module. `Request` accesses a website like a web browser would, and we can specify its behavior with `add_header`. We can then use `urlopen` to store the data from that website in a variable. If we access a .csv file this way, we can then format it into a pandas dataframe. In this method, the .csv file is never stored locally or on the Colab runtime, it is only used to generate the dataframe.

In [ ]:
from urllib.request import Request, urlopen

In [ ]:
req = Request('http://files.direxionfunds.com/DirexionWebsiteFiles/holdings_moon.csv')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)
df1 = pd.read_csv(content, skiprows=5)
df1.head()

We can also attempt to access files directly using pandas, for instance by using `pd.read_excel` to access an online .xlsx file.

In [ ]:
df2 = pd.read_excel('https://www.ssga.com/us/en/institutional/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-cnrg.xlsx', skiprows=4)
df2.head()

## Examining and cleaning data

Now let's take a look at the data we've obtained. We will work with the `weather_df` dataframe. Often times, we just want to inspect some of our data to make sure that it was loaded as we want. One way to get a sample of the data is to use the `.head()` function to print out the first few rows of the dataframe.

In [ ]:
weather_df.head() # first 5 rows by default

In [ ]:
weather_df.head(10) # first 10 rows

Similarly, we can use `.tail()` to look at the last few rows.

In [ ]:
weather_df.tail()

Finally, we can also summarize the data with `.describe()`.

In [ ]:
weather_df.describe()

We may wish to change the formatting for some columns. `datetime` is a standardized way of representing date and time. Let's take our `'Date Time'` column and standardize its formatting. To do so, we use the `pd.to_datetime` function on the desired column, then assign the result to the column. Take a moment to ensure you understand this methodology of assigning a column based on itself.

In [ ]:
weather_df['Date Time'] = pd.to_datetime(weather_df['Date Time'], format='%d.%m.%Y %H:%M:%S')
weather_df.head()

One of the first things we want to do when importing data is ensure that we are not missing any values. We can check each value to see if it is `nan` with `.isnull()`, which will return a `True`/`False` for every value.

In [ ]:
weather_df.isnull()

Instead of going through one by one to see if there are any `True`s, we can use `.any()` to check for us!

In [ ]:
weather_df.isnull().any()

This dataset was pre-cleaned for us, so we see that it is not missing any values! Let's go ahead a delete a value so we can look at ways of rectifying it. 

In [ ]:
weather_df.iloc[1, 3] = np.nan
weather_df.head()

In [ ]:
weather_df.isnull().any()

We must now decide what we want to do with rows/columns that are missing data. We can remove the row or column with the `.dropna()` function.

In [ ]:
weather_df_dropped_row = weather_df.dropna(axis=0)
weather_df_dropped_row.head()

In [ ]:
weather_df_dropped_column = weather_df.dropna(axis=1)
weather_df_dropped_column.head()

Oftentimes, however, we want to keep a data entry even if is incomplete (i.e. there are `NaN`s present). To do so, we can use `.ffill()` or `.bfill()` to fill in the missing data point with the previous or next data point, respectively.

In [ ]:
weather_df_forward_fill = weather_df.ffill()
weather_df_forward_fill.head()

In [ ]:
weather_df_back_fill = weather_df.bfill()
weather_df_back_fill.head()

Alternatively, we can fill in the point using interpolation, looking and neighboring points to guess what the missing point is.

In [ ]:
weather_df_interp = weather_df.interpolate()
weather_df_interp.head()

Finally, we can also choose a new index for our dataframe.

In [ ]:
weather_df = weather_df_interp.set_index('Date Time')
weather_df.head()

## Slicing and combining dataframes

Sometimes we are interested in only a portion of our dataset, and other times we have multiple datasets that we would like to combine into one. This is where slicing and combining come into play, respectively.

Let's look at the first day of data and only look at pressure (`p (mbar)` and temperature `T (degC)`).



In [ ]:
import datetime as dt # we want to create a datetime object outside of the dataframe
end_time = dt.datetime(2009, 1, 1, 1, 0, 0) # 2009-01-01 01:00:00

In [ ]:
first_slice = weather_df.loc[weather_df.index <= end_time, ['p (mbar)', 'T (degC)']]

In [ ]:
first_slice

For a slice of the second day, we need a start and end time. The easiest way to use multiple inequalities is with `np.logical_and()`.

In [ ]:
end_time_2 = dt.datetime(2009, 1, 1, 2, 0, 0)

In [ ]:
second_slice = weather_df.loc[np.logical_and(weather_df.index > end_time, weather_df.index <= end_time_2), ['p (mbar)', 'T (degC)']]

In [ ]:
second_slice

Now let's make things a bit more interesting and make a slice that overlaps with the first day, but also contains data from the second day. Let us also include `Tpot (K)` in this slice, but not include `p (mbar)`.

In [ ]:
overlap_start_time = dt.datetime(2009, 1, 1, 0, 30, 0)
overlap_end_time = dt.datetime(2009, 1, 1, 1, 30, 0)

In [ ]:
overlap_slice = weather_df.loc[np.logical_and(weather_df.index > overlap_start_time, weather_df.index <= overlap_end_time), ['T (degC)','Tpot (K)']]

In [ ]:
overlap_slice

### `merge()`

`merge()` allows us to combine dataframes with a choice `how`. We have a number of options:

*   `'inner'` - only include rows where there is data for every column
*   `'outer'` - include any row for which we have data
*   `'left'` - include any row for which there is data in the first ('left') dataframe
*   `'right'` - include any row for which there is data in the second ('right') dataframe

Note that for any columns that are present in both dataframes, they are kept separate when using `.merge()` **if** we pass `left_index=True, right_index=True`. See further below for how to merge columns where possible.



In [ ]:
df_merge_inner = pd.merge(first_slice, overlap_slice, left_index=True, right_index=True, how='inner')
df_merge_inner

In [ ]:
df_merge_outer = pd.merge(first_slice, overlap_slice, left_index=True, right_index=True, how='outer')
df_merge_outer

In [ ]:
df_merge_left = pd.merge(first_slice, overlap_slice, left_index=True, right_index=True, how='left')
df_merge_left

In [ ]:
df_merge_right = pd.merge(first_slice, overlap_slice, left_index=True, right_index=True, how='right')
df_merge_right

Note, we have been including the indices (`left_index=True, right_index=True`) which keeps the columns distinct. If we want to let Pandas merge columns where possible, we can use the default `left_indext=False, right_index=False`. First, we will reset the indices so that 'Date Time' data is retained.

In [ ]:
first_slice_no_index = first_slice.reset_index()
first_slice_no_index

In [ ]:
overlap_slice_no_index = overlap_slice.reset_index()
overlap_slice_no_index

In [ ]:
df_merge_inner_no_index = pd.merge(first_slice_no_index, overlap_slice_no_index, left_index=False, right_index=False, how='inner')
df_merge_inner_no_index

In [ ]:
df_merge_outer_no_index = pd.merge(first_slice_no_index, overlap_slice_no_index, left_index=False, right_index=False, how='outer')
df_merge_outer_no_index

### .append()

`.append()` simply sticks one dataframe onto the end of another. We can include the index or not, but it doesn't make a difference of how the dataframes are stuck together (both end up with 12 rows in this case).

In [ ]:
df_append_with_index = first_slice.append(overlap_slice, ignore_index=False)
df_append_with_index

In [ ]:
df_append_no_index = first_slice.append(overlap_slice, ignore_index=True)
df_append_no_index

### concat()

Again we have the options of `'outer'` which will include any columns which contain data, and `'inner'` which will only include columns which have data in all dataframes.

In [ ]:
df_concat_outer = pd.concat([first_slice, overlap_slice], join='outer')
df_concat_outer

In [ ]:
df_concat_inner = pd.concat([first_slice, overlap_slice], join='inner')
df_concat_inner

### `.combine_first`

`.combine_first` is used when we have a one dataframe that has priority over the other. This way, if both dataframes have the data for a given element (row-column pair), the data from the first dataframe will be retained.

This is *generally* my preferred way of combining dataframes, as it sets a clear priority if there are conflicting values, while also resulting in the most complete dataset possible. That said, the method of choice will depend on the problem at hand!

In [ ]:
df_combine_first = first_slice.combine_first(overlap_slice)
df_combine_first

A more illustrative example of `.combine_first`:

In [ ]:
s = pd.DataFrame([1, 2, np.nan])
t = pd.DataFrame([np.nan, 1, 3, 4])
print(s)
print(' ')
print(t)
print(' ')
print(s.combine_first(t))

### `.update()`

`.update()` behaves a bit differently, as it always modifies the dataframe in-place. It will overwrite any data in the first dataframe with data from the second dataframe, but will retain the same rows/columns as the first dataframe

In [ ]:
first_slice_updated = first_slice.copy()
first_slice_updated.update(overlap_slice)
first_slice_updated

In [ ]:
print(s)
print(t)
s_updated = s.copy()
s_updated.update(t)
print(s_updated)

# Plotting

It is often useful to have a visual representation of your data, e.g. via plotting. The most commonly used library for plotting with Python is matplotlib.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white' # sets the default background color to white

The most simple way to plot is `plt.plot(x_vals, y_vals)`

In [ ]:
plt.plot([0, 1, 2], [0, 5, 4])

Plotting a dataframe column will select the index for the x values if not specified.

In [ ]:
plt.plot(first_slice['T (degC)'])

However, a good plot should label its axes and have a title and legend.

In [ ]:
plt.plot(first_slice['T (degC)'], label='T (degC)', color='r')
plt.xlabel('Date Time')
plt.ylabel('T (degC)')
plt.title('Temp vs Time')
plt.legend()
plt.show()

Much better. Now, what if we want to have multiple plots as part of the same figure?

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))

ax.plot(first_slice['T (degC)'], color='r', label='T (degC)')
ax.tick_params(axis='y', colors='r')
ax.set_xlabel('Date time', size=20) 
ax.set_ylabel('Temp', size=14, color='r')
ax.set_title('Temp and Pressure, First Hour', size=24)

ax.axhline(-8.3, color="gray") # we can add an arbitrary horizontal line

ax2 = ax.twinx()
ax2.plot(first_slice['p (mbar)'], color='b', label='p (mbar)')
ax2.spines["right"].set_color('b')
ax2.tick_params(axis='y', colors='b')
ax2.set_ylabel('Pressure', color='b', size=14)

ax2.text(first_slice.index[-1], first_slice['p (mbar)'][-1], first_slice['p (mbar)'][-1], fontsize=15, color='b', horizontalalignment='center')
fig.patch.set_facecolor('w')

plt.show()

That's all for Lesson 3! In our next and final lesson, we will examine flow control and functions!

In the meantime, check out Quiz 3 on Canvas!